# Test run on building an agent
Features
- [ ] Agent can output a json file with the message for the user and the json data object
- [ ] Chat agent has history that will be sent back to the client
- [ ] Agent can build todo list items

In [38]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [39]:
from langchain.agents import tool


@tool
def get_word_length(word: str) -> int:
    """Returns the length of the input word."""
    return len(word)

tools = [get_word_length]
get_word_length.invoke("How are you today?")

18

In [40]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but don't know current events",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [41]:
llm_with_tools = llm.bind_tools(tools)

In [42]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)

from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

agent = (
    {
        "input": lambda x: x['input'],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x['intermediate_steps']
        )
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [43]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [44]:
list(agent_executor.stream({"input": "how many words are in this sentence?"}))



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'how many words are in this sentence?'}`


36There are 36 words in the sentence "how many words are in this sentence?".

> Finished chain.


[{'actions': [OpenAIToolAgentAction(tool='get_word_length', tool_input={'word': 'how many words are in this sentence?'}, log="\nInvoking: `get_word_length` with `{'word': 'how many words are in this sentence?'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_BZoOD7Ob8MTAMjKBthhssvUu', 'function': {'arguments': '{"word":"how many words are in this sentence?"}', 'name': 'get_word_length'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls'})], tool_call_id='call_BZoOD7Ob8MTAMjKBthhssvUu')],
  'messages': [AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_BZoOD7Ob8MTAMjKBthhssvUu', 'function': {'arguments': '{"word":"how many words are in this sentence?"}', 'name': 'get_word_length'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls'})]},
 {'steps': [AgentStep(action=OpenAIToolAgentAction(tool='get_word_length', tool_input={'word': 'how many words are 

In [45]:
llm.invoke("How many words are in this sentence?")

AIMessage(content='There are seven words in this sentence.', response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 15, 'total_tokens': 23}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': None})

## Adding memory

In [46]:
from langchain.prompts import MessagesPlaceholder

memory_key = "chat_history"
prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "You are very powerful assistant, but bad at calculating lengths of words.",
    ),
    MessagesPlaceholder(variable_name=memory_key),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

In [47]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

In [50]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
input1 = "how many letters in the word educa?"
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=input1),
        AIMessage(content=result["output"]),
    ]
)

In [ ]:
input2 = "how many letters in the word education?"
result2 = agent_executor.invoke({"input": input2, "chat_history": chat_history})
# use the extend method everytime you want to add a message to the chat history
chat_history.extend(
    [
        HumanMessage(content=input2),
        AIMessage(content=result["output"]),
    ]
)

In [74]:
def inovoke_and_append(input: str, chat_history: list):
    result = agent_executor.invoke({"input": input, "chat_history": chat_history})
    chat_history.extend(
        [
            HumanMessage(content=input),
            AIMessage(content=result["output"]),
        ]
    )
    return result

In [75]:
result3 = inovoke_and_append("how many letters in the word omgronnyie?", chat_history)



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'omgronnyie'}`


10The word "omgronnyie" has 10 letters.

> Finished chain.


In [87]:
type(result3['chat_history'][0])
type(result3['chat_history'][1])
type(result3['chat_history'][1]) == AIMessage

True

## Questions on appending messages
Would I need to append on both fronts to keep the cost down?
- ie append on the backend to keep it in track if the user reloads the data
- keep part of it on the frontend to only send back the answer instead of a large payload? 


How would I get the data initially to the frontend on new tab load or reload of webpage? 


How large is the chat and how expensive is it to send over the web? 


well have to test that.

[Message history with runnable](https://python.langchain.com/docs/expression_language/how_to/message_history)
- This is a way to store and retrieve conversation history. How would the data persist in the database?

In [63]:
result2['chat_history']

[HumanMessage(content='how many letters in the word educa?'),
 AIMessage(content='The word "educa" has 5 letters.'),
 HumanMessage(content='how many letters in the word education?'),
 AIMessage(content='The word "educa" has 5 letters.')]

In [64]:
result2

{'input': 'how many letters in the word education?',
 'chat_history': [HumanMessage(content='how many letters in the word educa?'),
  AIMessage(content='The word "educa" has 5 letters.'),
  HumanMessage(content='how many letters in the word education?'),
  AIMessage(content='The word "educa" has 5 letters.')],
 'output': 'The word "education" has 9 letters.'}